In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re


driver = webdriver.Firefox()
driver.get('https://www.wenku8.net/modules/article/reader.php?aid=2509')
page_html = driver.page_source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_html, 'html.parser')
tables = soup.find_all('table')

In [2]:
tables = soup.find_all('table')
import pandas as pd
# 將每個表格轉換為 DataFrame 並存儲在列表中
dataframes = []
for table in tables:
    df = pd.read_html(str(table))[0]
    dataframes.append(df)


dataframes[0].iloc[0,1]

C:\Users\bowli\AppData\Local\Temp\ipykernel_27248\2929733660.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


'第一卷 过去的恋情不肯结束'

In [3]:
dataframes = []
link_dataframes = []
for table in tables:
    df = pd.read_html(str(table))[0]
    dataframes.append(df)
    # 以下開始創造另外一個 table 與前面一個 list 相同格式
    links = []
    for row in table.find_all('tr'):
        link_row = []
        for cell in row.find_all(['td', 'th']):
            link = cell.find('a')
            if link and link.has_attr('href'):
                link_row.append(link['href'])
            else:
                link_row.append(None)
        links.append(link_row)
            
        # 創建包含鏈接的 DataFrame
    link_df = pd.DataFrame(links)
    link_dataframes.append(link_df)
title = soup.find('div', attrs={'id':'title'})
'''上面已完成階段性任務，基本上只有一個 table

datas frames 才有意義

如果整列的 title/ element 相同- 代表為卷名， 需要另外存一個資料夾


'''

C:\Users\bowli\AppData\Local\Temp\ipykernel_27248\3214481958.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


'上面已完成階段性任務，基本上只有一個 table\n\ndatas frames 才有意義\n\n如果整列的 title/ element 相同- 代表為卷名， 需要另外存一個資料夾\n\n\n'

In [ ]:
import os
import time
import random

mother_path = os.path.join( './data' , title.text )
os.makedirs( mother_path , exist_ok=True)

data_list = dataframes[0].values.tolist()
href_list = link_dataframes[0].values.tolist()
chapter_count = 0 
for idx_row,  row_list in enumerate( data_list ):
    if(row_list[0]==row_list[1] ):
        state_path = os.path.join( mother_path , f'{chapter_count}_' + row_list[0] )
        os.makedirs( state_path , exist_ok=True)
        #確定是第幾卷的內容
        count = 0
        chapter_count +=1
    else: #要進入網站讀取資料
        for idx_col, col_text in enumerate(row_list):
            try:
                driver.get(href_list[idx_row][idx_col])
                page_html = driver.page_source
                soup = BeautifulSoup(page_html, 'html.parser')
                content_text = soup.find('div', attrs={'id':'content'})
                temp_text = content_text.text
                pattern = r'\\(?!n)'
                replaced_text = re.sub(pattern, ' ', temp_text)
                file_path = os.path.join( state_path , f'{count}_'+col_text + '.txt')
                with open(file_path,'w', encoding='UTF-8') as file:
                    file.write(replaced_text)
                time.sleep( random.randint( 2 , 10))
                count += 1
            except:
                pass